In [ ]:
import numpy as np
import pandas as pd
import serial
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import time
import pymysql
from datetime import datetime

# ===============================
# 1. MySQL DB 연결 정보
# ===============================
DB_HOST = "localhost"
DB_PORT = 3306
DB_USER = "root"
DB_PASS = "12345"
DB_NAME = "electric_db"
TABLE_HISTORY = "전력량"
TABLE_REALTIME = "시간"

def get_connection():
    return pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME,
        charset="utf8"
    )

# ===============================
# 2. 과거 데이터 불러오기 (경계값 계산)
# ===============================
conn = get_connection()
df = pd.read_sql_query(
    "SELECT 시간, 전력량 FROM 전력량 ORDER BY 시간 ASC;", 
    conn
)
conn.close()

df.columns = ['date', 'price']
df["price"] = pd.to_numeric(df["price"], errors="coerce")  # 숫자 변환
df = df.dropna(subset=["price"])  # NaN 제거

if df["price"].empty:
    raise ValueError("오류: 'price' 열에 데이터가 없습니다.")

scaler = MinMaxScaler()
scaler.fit(df["price"].values.reshape(-1, 1))

boundaries = {
    "5등급": np.percentile(df["price"], 25),
    "6등급": np.percentile(df["price"], 10),
    "7등급": np.percentile(df["price"], 5),
    "3등급": np.percentile(df["price"], 75),
    "2등급": np.percentile(df["price"], 90),
    "1등급": np.percentile(df["price"], 95)
}

def classify_price(price):
    if price >= boundaries["상위 5%"]:
        return "1등급"
    elif price >= boundaries["상위 10%"]:
        return "2등급"
    elif price >= boundaries["상위 25%"]:
        return "3등급"
    elif price <= boundaries["하위 5%"]:
        return "7등급"
    elif price <= boundaries["하위 10%"]:
        return "6등급"
    elif price <= boundaries["하위 25%"]:
        return "5등급%"
    else:
        return "4등급"

# ===============================
# 3. LSTM 모델 로드
# ===============================
MODEL_PATH = "electricity_lstm_model.h5"

try:
    model = load_model(MODEL_PATH, compile=False)  # 최신 Keras 호환
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    print("✅ 모델 로드 성공!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("새 모델을 생성합니다...")
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(10, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    print("✅ 새 모델 생성 완료")

# ===============================
# 4. 시리얼 포트 설정 (아두이노 연결)
# ===============================
ser = serial.Serial("/dev/ttyUSB0", 9600, timeout=1)

SEQ_LEN = 10
recent_data = []

print("실시간 전력 요금 예측 + MySQL 저장 시작... (Ctrl+C 종료)")

try:
    while True:
        line = ser.readline().decode().strip()
        if not line:
            continue

        try:
            usage_kwh = float(line)
        except ValueError:
            continue

        price = usage_kwh * 100  # 단가 100원/kWh
        recent_data.append(price)

        if len(recent_data) >= SEQ_LEN:
            seq_input = np.array(recent_data[-SEQ_LEN:]).reshape(-1, 1)
            seq_scaled = scaler.transform(seq_input)
            seq_scaled = np.expand_dims(seq_scaled, axis=0)

            pred_scaled = model.predict(seq_scaled, verbose=0)
            pred_price = scaler.inverse_transform(pred_scaled)[0][0]

            category = classify_price(pred_price)

            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"[{now}] 예측 요금: {pred_price:,.0f}원 → {category}")

            conn = get_connection()
            cur = conn.cursor()
            insert_sql = f"""
                INSERT INTO {TABLE_REALTIME} (timestamp, usage_kwh, predicted_price, category)
                VALUES (%s, %s, %s, %s)
            """
            cur.execute(insert_sql, (now, usage_kwh, pred_price, category))
            conn.commit()
            conn.close()

        time.sleep(1)

except KeyboardInterrupt:
    print("종료합니다.")


✅ 모델 로드 성공!


C:\Users\Admin\AppData\Local\Temp\ipykernel_15692\1796778326.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


SerialException: could not open port '/dev/ttyUSB0': FileNotFoundError(2, '지정된 파일을 찾을 수 없습니다.', None, 2)